In [2]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

%matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-29 23:16:43,335 - utils - NumExpr defaulting to 4 threads.


## Ray tracing

Requisites:
* Raw files (blensor ouputs) shoul be placed in the root of the directory
* Raw files should be named `s1*.npy`, `s2*.npy`, etc
* Create a file with sensor positions in the root with name `scanner_pos.txt`. It should contain 4 columns:
    * sensor name i.e. `s1`, `s2`, etc
    * x coordinate
    * y coordinate
    * z coordinate

In [3]:
#
mockname = 'single_57_combined_lite'
debug = False
# downsample_debug = None
downsample = None
voxel_size = 0.13

m3att, meshfile, bia, voxk, lia, ws, resdir, tree = lad.get_lad20(mockname, debug=debug, 
                                downsample=downsample, voxel_size=voxel_size)

/home/omar/myproj/planttech/data/single_57_combined_lite/mesh.ply
size LPC: 	 11717
size new LPC: 	 11717
Stage 1: 0.005713362999999916
Mean distance between points: 	 0.005459734314491446
Stage 2: 0.19427350100000051
max --> [72, 74, 45]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (73, 75, 46)
ray tracker voxel dimensions: 	 (73, 75, 46)
Stage 3: 1.2047303250000008
Number of voxels ocupied by points cloud: 	 7088
Number of voxels ocupied by beam points cloud: 	 236320
Total number of voxels in plant regions: 	 251850
Number of voxels with attribute 1: 	 7088
Number of voxels with attribute 2: 	 229232
Number of voxels with attribute 3: 	 15530
Stage 4: 0.04232844999999941
Stage 5: 0.598852044


In [21]:
show=False
klia=False
use_true_lia=True

alphas_k = lad.alpha_k(bia, voxk, lia, ws, resdir, meshfile, 
                        show=show, klia=klia, use_true_lia=use_true_lia)

In [22]:

kbins = 2
# lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
# lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1)
lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0)
# lads_w = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0, mean_weights_k=alphas_k[:,7])
kmax = m3att.shape[2]
lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

In [23]:
fig = plt.figure(figsize=(4, 6))
# plt.plot(lads_min[0:,1], lads_min[0:,0], marker='*', label='Estimated min')
# plt.plot(lads_max[0:,1], lads_max[0:,0], marker='*', label='Estimated max')
plt.plot(lads_mesh[0:,1], lads_mesh[0:,0], marker='s', c='k', label='Truth')
plt.plot(lads_mid[0:,1], lads_mid[0:,0], marker='*', label='Estimated mid')
plt.plot(lads_0[0:,1], lads_0[0:,0], marker='*', label='Estimated 0')
# plt.plot(lads_w[0:,1], lads_w[0:,0], marker='*', label='Estimated weighted')

plt.xlabel(r'LAD ($m^2/m^3$)')
plt.ylabel(r'Height ($m$)')
# plt.grid()
plt.legend()
# plt.show()
# plt.xlim(-2,2)

mockdir = os.path.join(_data, mockname)
resdir_name = '%s_%s' %('results', mockname)
resdir = os.path.join(mockdir, resdir_name)
savefig = os.path.join(resdir, 'LADs.png')
plt.savefig(savefig, dpi=200, bbox_inches='tight')

# Dev Zone...

In [101]:
# tree.shape
points = tree.T[5:8].T
pcd = lad.points2pcd(points)
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

In [102]:
dist = []
for i in range(len(pcd.points)):
    [k, idx, d2] = pcd_tree.search_knn_vector_3d(pcd.points[i], 2)
    # print(pcd.points[i], pcd.points[idx[0]], d2[0])
    dist.append(d2[1])


In [103]:
fig = plt.figure(figsize=(10,6))
plt.hist(np.array(dist), 40)
mean = np.mean(np.array(dist))
plt.axvline(mean, lw=2, ls='--', c='k', label=mean)
plt.legend()
plt.show()